In [ ]:
import torch
import matplotlib.pyplot as plt
import os 

# Changing the directory to the main directory in the docker image
main_path = '/workspaces' 
os.chdir(main_path + '/ssa-solvers') # project directory

%load_ext line_profiler
%matplotlib inline

torch.set_default_tensor_type(torch.FloatTensor)

device = torch.device('cuda:0') if torch.cuda.is_available else torch.device("cpu")


In [ ]:
from ssa_solvers.simulators import StochasticSimulator, DeterministicSimulator 

end_time = 250
n_steps = 100
n_traj = 100
time_grid = torch.arange(0, end_time, int(end_time / n_steps), device=device)


### Simulating in cis mRNA - sRNA  circuit

In [ ]:
from circuits.mrna_srna.mrna_srna_incis import mRNAsRNAInCis, cfg

reaction_system_incis = mRNAsRNAInCis(device=device)
ode_simulator = DeterministicSimulator(
    reaction_system=reaction_system_incis,
    cfg=cfg
)
ssa_simulator_incis = StochasticSimulator(
    reaction_system=reaction_system_incis,
    cfg=cfg,
    device=device
)
reaction_system_incis.params = {'beta_fmrna': 2}  # increasing fmRNA production rate
init_pops = torch.zeros((reaction_system_incis.n_species, ), dtype=torch.int64, device=device) #torch.randint(1, (reaction_system.n_species, ), device=device)
ssa_simulator_incis.simulate(init_pops=init_pops, end_time=end_time, n_trajectories=n_traj)
ode_res_incis = ode_simulator.simulate(init_pops=init_pops, time_grid=time_grid)

### Simulating in trans mRNA - sRNA  circuit

In [ ]:
from circuits.mrna_srna.mrna_srna_intrans import mRNAsRNAInTrans, cfg

reaction_system_intrans = mRNAsRNAInTrans(device=device)
ode_simulator = DeterministicSimulator(
    reaction_system=reaction_system_intrans,
    cfg=cfg
)
cfg['stochastic_sim_cfg']['solver'] = 'first_reaction'
cfg['stochastic_sim_cfg']['save_to_file'] = True

ssa_simulator_intrans = StochasticSimulator(
    reaction_system=reaction_system_intrans,
    cfg=cfg,
    device=device
)
init_pops = torch.zeros((reaction_system_intrans.n_species, ), dtype=torch.int64, device=device) #torch.randint(1, (reaction_system.n_species, ), device=device)
ssa_simulator_intrans.simulate(init_pops=init_pops, end_time=end_time, n_trajectories=n_traj)
ode_res_intrans = ode_simulator.simulate(init_pops=init_pops, time_grid=time_grid)

### Comptuting the statistics on a grid

In [ ]:
means_incis, stds_incis = ssa_simulator_incis.data_set.mean_and_std(time_grid=time_grid)

means_intrans, stds_intrans = ssa_simulator_intrans.data_set.mean_and_std(time_grid=time_grid) 

### Plotting results

In [ ]:
time_grid = time_grid.cpu()

plt.figure()
species_idx_incis = 1
plt.plot(time_grid, means_incis[species_idx_incis, :], 'b', label='Mean SSA (in cis)')
plt.fill_between(time_grid, means_incis[species_idx_incis,:]+stds_incis[species_idx_incis, :], means_incis[species_idx_incis,:]-stds_incis[species_idx_incis, :], color='b', alpha=0.3)
plt.plot(time_grid, ode_res_incis[species_idx_incis, :], 'b--', label='ODE (in cis)')
plt.xlim([0, end_time])   
plt.xlabel('Time (s)')
plt.ylabel('Number of species')


species_idx_intrans = 2
plt.plot(time_grid, means_intrans[species_idx_intrans, :], 'r', label='Mean SSA (in trans)')
plt.fill_between(time_grid, means_intrans[species_idx_intrans,:]+stds_intrans[species_idx_intrans, :], means_intrans[species_idx_intrans,:]-stds_intrans[species_idx_intrans, :], 
                color='r',alpha=0.3)
plt.plot(time_grid, ode_res_intrans[species_idx_intrans, :], 'r--', label='ODE (in trans)')
plt.xlim([0, end_time])    
plt.xlabel('Time (s)')
plt.ylabel('Number of species')
plt.xlim([0, max(time_grid)])
plt.ylim([0, 1.1 * max([(means_intrans[species_idx_intrans,:]+stds_intrans[species_idx_intrans, :]).max(), 
                        (means_incis[species_idx_incis,:]+stds_incis[species_idx_incis, :]).max()])])
plt.legend()
